<a href="https://colab.research.google.com/github/Anil-matcha/openai-functions/blob/main/Openai_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install openai
!pip install duckpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 6.1 MB/s eta 0:00:00


In [36]:
import openai
import json
from duckpy import Client
import ast

duckduckgo_client = Client()

openai.api_key = "api-key"

def search_api(input):
    output = duckduckgo_client.search(input)
    return str(output)

def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "Tell me IPL 2023 winner?"}],
        functions=[
            {
                "name": "search_api",
                "description": "Used to search online",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "input": {
                            "type": "string",
                            "description": "Input query",
                        },
                    },
                    "required": ["input"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        args = ast.literal_eval(message["function_call"]["arguments"])
        print("Message", args, type(args))
        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = search_api(
            input=args.get("input"),
        )

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "Tell me IPL 2023 winner?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print(run_conversation())

Message {'input': 'IPL 2023 winner'} <class 'dict'>
{
  "id": "chatcmpl-7RM8RxJRnl3k4fOTBN9RWO7521S6h",
  "object": "chat.completion",
  "created": 1686754299,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The winner of the IPL 2023 was the Chennai Super Kings (CSK). They defeated the Gujarat Titans (GT) to claim their fifth IPL title."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 3412,
    "completion_tokens": 32,
    "total_tokens": 3444
  }
}
